In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Titanic SQL Example") \
    .getOrCreate()

### Read the data first

In [2]:
spark.read.csv("data/Titanic_data.csv")

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string]

In [3]:
t_raw_data = spark.read.csv("data/Titanic_data.csv",header=True,inferSchema=True,)
t_raw_data

DataFrame[id: int, PClass: string, Age: double, Gender: string, Survived: int, GenderCode: int]

### Start exploring the data

In [4]:
# See how the data looks like
t_raw_data.show()
# Pclass: Passenger Class
# Survived either 1 or 0
# GenderCode 1 for female, 0 for male

+---+------+----+------+--------+----------+
| id|PClass| Age|Gender|Survived|GenderCode|
+---+------+----+------+--------+----------+
|  1|   1st|29.0|female|       1|         1|
|  2|   1st| 2.0|female|       0|         1|
|  3|   1st|30.0|  male|       0|         0|
|  4|   1st|25.0|female|       0|         1|
|  5|   1st|0.92|  male|       1|         0|
|  6|   1st|47.0|  male|       1|         0|
|  7|   1st|63.0|female|       1|         1|
|  8|   1st|39.0|  male|       0|         0|
|  9|   1st|58.0|female|       1|         1|
| 10|   1st|71.0|  male|       0|         0|
| 11|   1st|47.0|  male|       0|         0|
| 12|   1st|19.0|female|       1|         1|
| 13|   1st| 0.0|female|       1|         1|
| 14|   1st| 0.0|  male|       1|         0|
| 15|   1st| 0.0|  male|       0|         0|
| 16|   1st|50.0|female|       1|         1|
| 17|   1st|24.0|  male|       0|         0|
| 18|   1st|36.0|  male|       0|         0|
| 19|   1st|37.0|  male|       1|         0|
| 20|   1s

In [5]:
t_raw_data.count()

1313

In [6]:
# Print the schema
t_raw_data.printSchema()

root
 |-- id: integer (nullable = true)
 |-- PClass: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- GenderCode: integer (nullable = true)



In [7]:
# Describe the data
t_raw_data.describe().show()

+-------+-----------------+------+------------------+------+-------------------+-------------------+
|summary|               id|PClass|               Age|Gender|           Survived|         GenderCode|
+-------+-----------------+------+------------------+------+-------------------+-------------------+
|  count|             1313|  1313|              1313|  1313|               1313|               1313|
|   mean|            657.0|  null|17.502574257425742|  null| 0.3427265803503427| 0.3518659558263519|
| stddev|379.1747618183468|  null| 18.51694476332819|  null|0.47480181908910607|0.47773437008438874|
|    min|                1|     *|               0.0|female|                  0|                  0|
|    max|             1313|   3rd|              71.0|  male|                  1|                  1|
+-------+-----------------+------+------------------+------+-------------------+-------------------+



In [8]:
t_names = spark.read.csv("data/Titanic_names.csv",header=True,inferSchema=True,)
t_names

DataFrame[id: int, Name: string]

### Sanitize the data 

In [13]:
# Try to remove the 0.0 from the age column
t_data = t_raw_data.filter(t_raw_data['age'] >0)
t_data.show()

+---+------+----+------+--------+----------+
| id|PClass| Age|Gender|Survived|GenderCode|
+---+------+----+------+--------+----------+
|  1|   1st|29.0|female|       1|         1|
|  2|   1st| 2.0|female|       0|         1|
|  3|   1st|30.0|  male|       0|         0|
|  4|   1st|25.0|female|       0|         1|
|  5|   1st|0.92|  male|       1|         0|
|  6|   1st|47.0|  male|       1|         0|
|  7|   1st|63.0|female|       1|         1|
|  8|   1st|39.0|  male|       0|         0|
|  9|   1st|58.0|female|       1|         1|
| 10|   1st|71.0|  male|       0|         0|
| 11|   1st|47.0|  male|       0|         0|
| 12|   1st|19.0|female|       1|         1|
| 16|   1st|50.0|female|       1|         1|
| 17|   1st|24.0|  male|       0|         0|
| 18|   1st|36.0|  male|       0|         0|
| 19|   1st|37.0|  male|       1|         0|
| 20|   1st|47.0|female|       1|         1|
| 21|   1st|26.0|  male|       1|         0|
| 22|   1st|25.0|  male|       0|         0|
| 23|   1s

In [14]:
t_data.count()

756

In [15]:
t_data.describe().show()

+-------+------------------+------+------------------+------+------------------+-------------------+
|summary|                id|PClass|               Age|Gender|          Survived|         GenderCode|
+-------+------------------+------+------------------+------+------------------+-------------------+
|  count|               756|   756|               756|   756|               756|                756|
|   mean| 520.5489417989418|  null|30.397989417989415|  null| 0.414021164021164|0.38095238095238093|
| stddev|318.61611186598935|  null|14.259048710359023|  null|0.4928782551360203| 0.4859424028848232|
|    min|                 1|   1st|              0.17|female|                 0|                  0|
|    max|              1313|   3rd|              71.0|  male|                 1|                  1|
+-------+------------------+------+------------------+------+------------------+-------------------+



### Start SQL Queries on the dataframe

In [16]:
# Create a temporary view in order to use SQL queries
t_data.createOrReplaceTempView('titanic_data')

In [17]:
# Show only survivors
survivors = spark.sql("SELECT * FROM titanic_data WHERE survived = 1")
survivors.show()
survivors.count()

+---+------+----+------+--------+----------+
| id|PClass| Age|Gender|Survived|GenderCode|
+---+------+----+------+--------+----------+
|  1|   1st|29.0|female|       1|         1|
|  5|   1st|0.92|  male|       1|         0|
|  6|   1st|47.0|  male|       1|         0|
|  7|   1st|63.0|female|       1|         1|
|  9|   1st|58.0|female|       1|         1|
| 12|   1st|19.0|female|       1|         1|
| 16|   1st|50.0|female|       1|         1|
| 19|   1st|37.0|  male|       1|         0|
| 20|   1st|47.0|female|       1|         1|
| 21|   1st|26.0|  male|       1|         0|
| 23|   1st|25.0|  male|       1|         0|
| 24|   1st|19.0|female|       1|         1|
| 25|   1st|28.0|  male|       1|         0|
| 27|   1st|39.0|  male|       1|         0|
| 28|   1st|30.0|female|       1|         1|
| 29|   1st|58.0|female|       1|         1|
| 31|   1st|45.0|female|       1|         1|
| 32|   1st|22.0|female|       1|         1|
| 37|   1st|44.0|female|       1|         1|
| 38|   1s

313

In [19]:
# Show only female survivors

survivors = spark.sql("SELECT * FROM titanic_data WHERE survived = 1 and Gender='female'")
survivors.show()
survivors.count()

+---+------+----+------+--------+----------+
| id|PClass| Age|Gender|Survived|GenderCode|
+---+------+----+------+--------+----------+
|  1|   1st|29.0|female|       1|         1|
|  7|   1st|63.0|female|       1|         1|
|  9|   1st|58.0|female|       1|         1|
| 12|   1st|19.0|female|       1|         1|
| 16|   1st|50.0|female|       1|         1|
| 20|   1st|47.0|female|       1|         1|
| 24|   1st|19.0|female|       1|         1|
| 28|   1st|30.0|female|       1|         1|
| 29|   1st|58.0|female|       1|         1|
| 31|   1st|45.0|female|       1|         1|
| 32|   1st|22.0|female|       1|         1|
| 37|   1st|44.0|female|       1|         1|
| 38|   1st|59.0|female|       1|         1|
| 39|   1st|60.0|female|       1|         1|
| 42|   1st|53.0|female|       1|         1|
| 43|   1st|58.0|female|       1|         1|
| 49|   1st|36.0|female|       1|         1|
| 50|   1st|14.0|female|       1|         1|
| 57|   1st|47.0|female|       1|         1|
| 59|   1s

217

In [27]:
# List different types of passenger classes

survivors = spark.sql("SELECT DISTINCT(pclass) FROM titanic_data")
survivors.show()

+------+
|pclass|
+------+
|   2nd|
|   1st|
|   3rd|
+------+



In [32]:
survivors = spark.sql("SELECT pclass, count(pclass) FROM titanic_data GROUP BY pclass")
survivors.show()

+------+-------------+
|pclass|count(pclass)|
+------+-------------+
|   2nd|          212|
|   1st|          226|
|   3rd|          318|
+------+-------------+



In [ ]:
# List number of people in each passenger class

In [ ]:
# List number of survivors in each passenger class

#### Using dataframe functions

In [33]:
# but use 't_raw_data' dataframe to compare with original data
t_raw_data.filter(t_raw_data.Survived == 1).groupby("pclass").count().show()

+------+-----+
|pclass|count|
+------+-----+
|   2nd|  119|
|   1st|  193|
|   3rd|  138|
+------+-----+



In [35]:
# display the list of number of people in each passenger class & list of number of survivors in each passenger class,
# as a single dataframe
dfclass_joined = dfclass_counts.join(dfclass_s_counts, "pclass")
dfclass_joined.show()

NameError: name 'dfclass_counts' is not defined

In [ ]:
dfclass_joined.select("pclass", dfclass_joined["pclass_survived_count"]/dfclass_joined["pclass_count"]).show()

#### Get the names

In [37]:
t_names.createOrReplaceTempView("titanic_names")
t_names.show()

+---+--------------------+
| id|                Name|
+---+--------------------+
|  1|Allen, Miss Elisa...|
|  2|Allison, Miss Hel...|
|  3|Allison, Mr Hudso...|
|  4|Allison, Mrs Huds...|
|  5|Allison, Master H...|
|  6|  Anderson, Mr Harry|
|  7|Andrews, Miss Kor...|
|  8|Andrews, Mr Thoma...|
|  9|Appleton, Mrs Edw...|
| 10|Artagaveytia, Mr ...|
| 11|Astor, Colonel Jo...|
| 12|Astor, Mrs John J...|
| 13|Aubert, Mrs Leont...|
| 14|Barkworth, Mr Alg...|
| 15|  Baumann, Mr John D|
| 16|Baxter, Mrs James...|
| 17|Baxter, Mr Quigg ...|
| 18| Beattie, Mr Thomson|
| 19|Beckwith, Mr Rich...|
| 20|Beckwith, Mrs Ric...|
+---+--------------------+
only showing top 20 rows



In [48]:
# Get the names of all males (IN)

survivors = spark.sql("SELECT name FROM titanic_names where name like '%Mr %'")
survivors.show()


+--------------------+
|                name|
+--------------------+
|Allison, Mr Hudso...|
|  Anderson, Mr Harry|
|Andrews, Mr Thoma...|
|Artagaveytia, Mr ...|
|Barkworth, Mr Alg...|
|  Baumann, Mr John D|
|Baxter, Mr Quigg ...|
| Beattie, Mr Thomson|
|Beckwith, Mr Rich...|
|Behr, Mr Karl Howell|
|  Birnbaum, Mr Jakob|
|Bishop, Mr Dickin...|
|Bjornstrm-Steffan...|
|Blackwell, Mr Ste...|
|     Blank, Mr Henry|
|Borebank, Mr John...|
|  Bradley, Mr George|
|Brady, Mr John Be...|
|   Brandeis, Mr Emil|
|Calderhead, Mr Ed...|
+--------------------+
only showing top 20 rows



In [ ]:
# List people and their ages given their ages is more than the average age 

In [ ]:
# Attach the names of people with the above result (use the id column to join)
t_data_with_names = ....

In [ ]:
# Partition the data by columns
t_data_with_names.write.partitionBy("gender", "survived").parquet("data/titanic_part.parquet")

In [ ]:
df_part = spark.read.parquet("data/titanic_part.parquet")

#### ----

In [ ]:
# show the age, age groups (counts) ordered by age


In [ ]:
# show the age, age groups (counts) ordered by age for different genders ordered by age


In [ ]:
# show the age, age groups (counts) ordered by age for different genders ordered by gender then age


### Adding new records

In [ ]:
from pyspark import Row
t_record_rdd = spark.sparkContext.parallelize([Row(id=0, Name="Jon Snow", PClass="2nd", Age=31.00, Gender="male", Survived=1, GenderCode=0)])
t_record_rdd.collect()

In [ ]:
t_record_df_unordered = t_record_rdd.toDF()
t_record_df_unordered.show()
t_record_df = t_record_df_unordered.select("id", "name", "pclass", "age", "gender", "survived", "gendercode")
t_record_df.show()

In [ ]:
# take a union of the above dataset with t_data_with_names